<a href="https://colab.research.google.com/github/JiaUF/OnlineQuantumLab/blob/main/Deutsch_Jozsa_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Before running the codes, please install qiskit in the Colab terminal by using the commands:  
> pip install qiskit  
> pip install 'qiskit[visualization]'  
> pip install qiskit-aer


# Deutsch-Jozsa Algorithm: Implementation in Qiskit

This notebook walks you through the Deutsch-Jozsa algorithm for determining whether a function $f(x)$ is constant or balanced.

In the Deutsch–Jozsa algorithm, we are given a special type of function $f(x)$ that takes a 1-bit input (either 0 or 1) and outputs either 0 or 1. We are promised that the function is one of the following:

- **Constant function**:
  The output is the same for all inputs. That is, $f(0) = f(1)$.
  Examples:
  - $f(0) = 0,\ f(1) = 0$
  - $f(0) = 1,\ f(1) = 1$

- **Balanced function**:
  The output is 0 for one input and 1 for the other. That is, $f(0) \ne f(1)$.
  Examples:
  - $f(0) = 0,\ f(1) = 1$
  - $f(0) = 1,\ f(1) = 0$

The goal of the Deutsch–Jozsa algorithm is to determine whether the unknown function is **constant** or **balanced**, using just **one** evaluation on a quantum computer. This highlights the power of quantum algorithms compared to classical ones.

The key idea is to use **quantum parallelism and interference** to determine the function type with just one query.

Now, let's explore how to implement the Deutsch-Jozsa algorithm using the Qiskit quantum simulator.



In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

## Step 1: Define the Oracle for the function $f(x)$

The oracle will implement the transformation:

$|x⟩|y⟩ \rightarrow |x⟩|y \oplus f(x)⟩$

We create oracles for constant and balanced functions.


In [21]:
# Return a quantum circuit implementing the DJ oracle for f(x).
# You can understand oracle as a black box that performs some computation.

def deutsch_jozsa_oracle(case='constant'):
    # We create a quantum circuit that has 2 qubits.
    oracle = QuantumCircuit(2)
    if case == 'balanced':
        # control-NOT gate. it applies a NOT (X) gate to the second qubit
        # (index 1) if and only if the first qubit (index 0) is in the state |1⟩
        oracle.cx(0, 1)
    elif case == 'constant':
        pass  # identity if f(x) = 0
    return oracle


## Step 2: Deutsch-Jozsa Circuit Construction

We initialize the qubits to $|0⟩$ and $|1⟩$, then apply Hadamard gates to both, apply the oracle, and apply another Hadamard to the first qubit.


In [13]:
def deutsch_jozsa_circuit(case='constant'):
    qc = QuantumCircuit(2, 1)

    # Initialize |0⟩|1⟩
    qc.x(1)  # Set bottom qubit to |1⟩
    qc.h([0, 1])  # Apply Hadamard to both

    # Apply the oracle
    oracle = deutsch_jozsa_oracle(case)
    qc.append(oracle.to_gate(), [0, 1])

    # Apply Hadamard to the first qubit
    qc.h(0)

    # Measure the first qubit
    qc.measure(0, 0)
    print(qc)

    return qc

## Step 3: Run the circuit and interpret results

If the result is always 0, then the function is **constant**.  
If the result is 1, then the function is **balanced**.


In [20]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def run_and_plot(case='constant'):
    # Get Deutsch-Jozsa circuit
    qc = deutsch_jozsa_circuit(case)

    # Choose backend simulator
    backend = Aer.get_backend('aer_simulator')

    # Transpile circuit for backend
    transpiled = transpile(qc, backend)

    # Run with memory enabled to access individual shots
    job = backend.run(transpiled, shots=1024, memory=True)
    result = job.result()

    # Access full shot memory
    memory = result.get_memory()
    print(f"[{case.upper()}] First 5 measurement outcomes:", memory[:5])

    # Access and plot measurement counts
    counts = result.get_counts()
    print(f"[{case.upper()}] Measurement counts:", counts)
    #plot_histogram(counts)
    #plt.title(f"Deutsch-Jozsa result for a {case} function")
    #plt.show()

# Run both test cases
run_and_plot('constant')
run_and_plot('balanced')

     ┌───┐     ┌──────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0             ├┤ H ├┤M├
     ├───┤┌───┐│  circuit-212 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1             ├──────╫─
     └───┘└───┘└──────────────┘      ║ 
c: 1/════════════════════════════════╩═
                                     0 
[CONSTANT] First 5 measurement outcomes: ['0', '0', '0', '0', '0']
[CONSTANT] Measurement counts: {'0': 1024}
     ┌───┐     ┌──────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0             ├┤ H ├┤M├
     ├───┤┌───┐│  circuit-217 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1             ├──────╫─
     └───┘└───┘└──────────────┘      ║ 
c: 1/════════════════════════════════╩═
                                     0 
[BALANCED] First 5 measurement outcomes: ['1', '1', '1', '1', '1']
[BALANCED] Measurement counts: {'1': 1024}


### Interpreting the Results

As you can see from the results, when the function $f(x)$ is **constant**, the measurement of qubit 0 always yields **0**.  
In contrast, when $f(x)$ is **balanced**, the measurement of qubit 0 always yields **1**.  

This clear distinction is what makes the Deutsch–Jozsa algorithm powerful—it determines the global property of a function (constant or balanced) with a single query.